In [1]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import time
import librairie_qui_remplace_open_cv as k
import ttkbootstrap as tb
from tkinter import filedialog
from PIL import Image, ImageTk
from tkinter import messagebox

In [ ]:
class ImageProcessorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Traitement d'Images - TtkBootstrap")
        self.root.geometry("900x650")

        self.image = None
        self.image2 = None
        self.processed_image = None
        self.param_entries = {}

        self.algorithm_requirements = {
            "Sobel": {"images": 1, "params": []},
            "Otsu": {"images": 1, "params": []},
            "Nagao": {"images": 1, "params": []},
            "Hough": {"images": 1, "params": []},
            "Transformation lut": {"images": 1, "params": []},
            "Niveaux de gris": {"images": 1, "params": []},
            "Rapport inverse contraste": {"images": 1, "params": []},
            "Étirement d'histogramme": {"images": 1, "params": []},
            "Égalisation d'histogramme": {"images": 1, "params": []},
            "Spécification d'histogramme": {"images": 2, "params": []},
            "Addition": {"images": 2, "params": []},
            "Soustraction": {"images": 2, "params": []},
            "&": {"images": 2, "params": []},
            "|": {"images": 2, "params": []},
            "Compression": {"images": 1, "params": []},
            "Médian": {"images": 1, "params": ["kernel_size"]},
            "Multiplier avec ratio": {"images": 1, "params": ["ratio"]},
            "Convolution": {"images": 1, "params": ["filtre"]},
        }

        self.setup_ui()

    def setup_ui(self):
        self.frame_top = tb.Frame(self.root)
        self.frame_top.pack(pady=10)

        self.btn_load = tb.Button(self.frame_top, text="Charger une image", command=self.load_image)
        self.btn_load.pack(side="left", padx=10)

        self.algorithm_var = tb.StringVar(value="Sobel")
        self.algorithms = list(self.algorithm_requirements.keys())
        self.dropdown = tb.Combobox(self.frame_top, textvariable=self.algorithm_var, values=self.algorithms, state="readonly")
        self.dropdown.pack(side="left", padx=10)
        self.dropdown.bind("<<ComboboxSelected>>", self.on_algorithm_change)

        self.btn_apply = tb.Button(self.frame_top, text="Appliquer", command=self.apply_algorithm)
        self.btn_apply.pack(side="left", padx=10)

        # Frame pour les images
        self.frame_images = tb.Frame(self.root)
        self.frame_images.pack(fill="both", expand=True)

        # Labels pour afficher les images
        self.label_original = tb.Label(self.frame_images, text="Image Originale")
        self.label_original.pack(side="left", padx=10)

        self.label_second = tb.Label(self.frame_images, text="Deuxième Image")
        self.label_second.pack(side="left", padx=10)

        self.label_result = tb.Label(self.frame_images, text="Résultat")
        self.label_result.pack(side="left", padx=10)

        # Frame pour les paramètres
        self.frame_params = tb.Frame(self.root)
        self.frame_params.pack(pady=10)

        self.on_algorithm_change()

    def on_algorithm_change(self, event=None):
        algo = self.algorithm_var.get()
        requirements = self.algorithm_requirements.get(algo, {})

        if requirements.get("images", 1) == 2:
            self.add_second_image_button()
        else:
            self.remove_second_image_button()

        for widget in self.frame_params.winfo_children():
            widget.destroy()
        self.param_entries = {}

        for param in requirements.get("params", []):
            self.create_param_input(f"Paramètre : {param}", param, 50)

    def add_second_image_button(self):
        if not hasattr(self, 'btn_load_second'):
            self.btn_load_second = tb.Button(self.frame_top, text="Charger 2ème image", command=self.load_second_image)
            self.btn_load_second.pack(side="left", padx=10)

    def remove_second_image_button(self):
        if hasattr(self, 'btn_load_second'):
            self.btn_load_second.destroy()
            del self.btn_load_second
            self.image2 = None

    def create_param_input(self, label_text, param_key, default_value):
        label = tb.Label(self.frame_params, text=label_text)
        label.pack(side="left", padx=10)

        entry = tb.Entry(self.frame_params)
        entry.insert(0, str(default_value))
        entry.pack(side="left", padx=10)

        self.param_entries[param_key] = entry

    def get_params(self):
        return {key: entry.get() for key, entry in self.param_entries.items()}

    def load_image(self):
        file_path = filedialog.askopenfilename()
        if file_path:
            self.image = k.lireImage(file_path)
            self.display_image(self.image, self.label_original)
            self.image = k.convertir_en_niveaux_de_gris(self.image)

    def load_second_image(self):
        file_path = filedialog.askopenfilename()
        if file_path:
            self.image2 = k.lireImage(file_path)
            self.display_image(self.image2, self.label_second)
            self.image2 = k.convertir_en_niveaux_de_gris(self.image2)

    def apply_algorithm(self):
        algo = self.algorithm_var.get()
        requirements = self.algorithm_requirements.get(algo, {"images": 1, "params": []})

        if requirements["images"] >= 1 and self.image is None:
            messagebox.showerror("Erreur", "Veuillez charger une image.")
            return

        if requirements["images"] == 2 and self.image2 is None:
            messagebox.showerror("Erreur", "Veuillez charger une deuxième image.")
            return

        params = self.get_params()

        if algo == "Sobel":
            self.processed_image = k.appliquer_filtre_sobel(self.image)
        elif algo == "Médian":
            self.processed_image = k.filtre_median(self.image)
        elif algo == "Otsu":
            self.processed_image = k.methode_otsu(self.image)[0]
        elif algo == "Nagao":
            self.processed_image = k.filtre_nagao(self.image)
        elif algo == "Hough":
            self.processed_image = k.detect_lines_avec_Hough(self.image)
        elif algo == "Transformation lut":
            self.processed_image = k.transformation_lut(self.image)
        elif algo == "Niveaux de gris":
            self.processed_image = k.convertir_en_niveaux_de_gris(self.image)
        elif algo == "Rapport inverse contraste":
            self.processed_image = k.rapport_inverse_contraste(self.image)
        elif algo == "Compression":
            self.processed_image = k.compression_image_bilineaire(self.image, 0.5)
        elif algo == "Étirement d'histogramme":
            self.processed_image = k.etirement_histog(self.image)
        elif algo == "Égalisation d'histogramme":
            self.processed_image = k.egalisation_histogramme(self.image)
        elif algo == "Spécification d'histogramme":
            self.processed_image = k.specification_histogramme(self.image, self.image2)
        elif algo == "Addition":
            self.processed_image = k.addition2images(self.image, self.image2)
        elif algo == "Soustraction":
            self.processed_image = k.soustraction2images(self.image, self.image2)
        elif algo == "&":
            self.processed_image = k.appliquerOperatorET(self.image, self.image2)
        elif algo == "|":
            self.processed_image = k.appliquerOperatorOU(self.image, self.image2)
        elif algo == "Médian":
            kernel_size = int(params.get("kernel_size", 3))
            self.processed_image = k.filtre_median(self.image, kernel_size)
        elif algo == "Convolution":
            #filtre = self.get_filter_matrix(params.get("filtre"))
            #self.processed_image = k.convolution(self.image, filtre)
            self.create_filter_grid(filter_size=3)
        elif algo == "Multiplier avec ratio":
            ratio = int(params.get("ratio"))
            self.processed_image = k.multiplier_image_avec_ratio(self.image, ratio)

        self.display_image(self.processed_image, self.label_result)

    def display_image(self, img, label):
        if img is None:
            return
        img = Image.fromarray(np.uint8(img))
        img = img.resize((300, 300))
        img_tk = ImageTk.PhotoImage(image=img)
        label.configure(image=img_tk)
        label.image = img_tk
        
    def create_filter_grid(self, filter_size=3):
        # Effacer les anciennes entrées (si elles existent)
        for widget in self.frame_params.winfo_children():
            widget.destroy()

        self.filter_entries = []  # Liste des entrées pour la grille de filtre

        # Créer les champs de saisie pour chaque élément du filtre
        for i in range(filter_size):
            row_entries = []  # Liste des entrées pour la ligne i
            for j in range(filter_size):
                entry = tb.Entry(self.frame_params, width=5)
                entry.grid(row=i, column=j, padx=5, pady=5)
                row_entries.append(entry)
            self.filter_entries.append(row_entries)

        # Ajouter un bouton pour valider le filtre
        self.btn_validate_filter = tb.Button(self.frame_params, text="Valider le filtre", command=self.get_filter_matrix_from_entries)
        self.btn_validate_filter.grid(row=filter_size, column=0, columnspan=filter_size, pady=10)


    def get_filter_matrix_from_entries(self):
        try:
            # Extraire les valeurs des champs Entry et les convertir en matrice numpy
            filter_matrix = np.array([[int(entry.get()) for entry in row] for row in self.filter_entries])

            # Afficher la matrice du filtre dans la console (optionnel)
            print("Filtre saisi :", filter_matrix)

            # Appliquer le filtre à l'image
            self.processed_image = k.convolution(self.image, filter_matrix)

            # Afficher l'image résultante
            self.display_image(self.processed_image, self.label_result)

        except ValueError:
            messagebox.showerror("Erreur", "Les valeurs du filtre ne sont pas valides.")


if __name__ == "__main__":
    root = tb.Window(themename="superhero")
    app = ImageProcessorApp(root)
    root.mainloop()